In [ ]:
# All your original imports plus rate limiting
import json
import requests
import pandas as pd
import chess.pgn
import io
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from scipy import stats
import chess.engine
import sys
import logging
import math
from enum import Enum
from time import sleep  # For rate limiting

# Configure logging
logging.basicConfig(
    level=logging.INFO, format="%(levelname)s: %(message)s", stream=sys.stdout
)

# Your existing configuration
sns.set(style="whitegrid")
GENERAL_PGN_FILE_PATH = "/Users/benjaminrosales/Desktop/Chess Study Materials & Data/Comparison Games/lichess_db_standard_rated_2017-05.pgn"
STOCKFISH_PATH = "/opt/homebrew/bin/stockfish"
ADHD_USERNAMES = ["teoeo", "Tobermorey", "apostatlet", "LovePump1000", "Stuntmanandy", 
                  "Banfy_B", "ChessyChesterton12", "Yastoon", "Timy1976", "SonnyDayz11", "xiroir"]

In [ ]:
# Elocator API Functions
def get_position_complexity_with_retry(fen, max_retries=3):
    """Get position complexity from Elocator API with retry logic"""
    for attempt in range(max_retries):
        try:
            response = requests.post(
                "https://elocator.fly.dev/complexity/",
                json={"fen": fen}
            )
            if response.status_code == 200:
                sleep(0.1)  # 100ms delay between requests
                return response.json()["complexity_score"]
            logging.warning(f"API returned status code {response.status_code}")
        except Exception as e:
            logging.warning(f"API request failed (attempt {attempt + 1}/{max_retries}): {e}")
        sleep(1)  # Wait longer between retries
    return None

def analyze_game_complexity(pgn):
    """Get full game analysis from Elocator API"""
    try:
        response = requests.post(
            "https://elocator.fly.dev/analyze-game/",
            json={"pgn": pgn}
        )
        return response.json() if response.status_code == 200 else None
    except Exception as e:
        logging.warning(f"Failed to analyze game complexity: {e}")
        return None

In [ ]:
# Your existing utility functions
[All your existing functions from the original notebook's second cell]

In [ ]:
# Modified process_games function with Elocator integration
def process_games(games, group_label, engine, max_depth=2):
    all_moves = []
    for game in tqdm(games, desc=f"Processing {group_label} games"):
        try:
            board = game.board()
            game_id = game.headers.get("Site", "Unknown")
            # [Your existing header extraction code]

            node = game
            move_number = 0
            prev_evaluation = None
            prev_time_remaining = None
            prev_winning_chances = None

            # Check for evaluations
            if not any("%eval" in n.comment for n in game.mainline()):
                continue

            while node.variations:
                next_node = node.variations[0]
                move = next_node.move
                
                # Get position complexity before making the move
                current_fen = board.fen()
                complexity_score = get_position_complexity_with_retry(current_fen)
                
                # [Your existing move processing code]
                
                move_data = {
                    # [Your existing move_data fields]
                    'ElocatorComplexity': complexity_score,  # Add complexity score
                    'PositionComplexity': position_complexity  # Your existing complexity
                }

                all_moves.append(move_data)
                
                # Update board and continue
                board.push(move)
                node = next_node
                prev_evaluation = evaluation
                prev_time_remaining = time_remaining
                prev_winning_chances = winning_chances

        except Exception as e:
            logging.error(f"Error processing game {game_id}: {e}")
            continue

    moves_df = pd.DataFrame(all_moves)
    return moves_df

In [ ]:
# Test the implementation with a small sample
logging.info("Testing Elocator integration with a small sample...")

# Fetch just a few games for testing
test_adhd_games = []
for username in ADHD_USERNAMES[:2]:  # Test with first 2 usernames
    logging.info(f"Fetching games for user '{username}'...")
    user_games = fetch_lichess_games(username, max_games=2)  # Just 2 games per user
    test_adhd_games.extend(user_games)

if test_adhd_games:
    # Initialize engine
    engine = chess.engine.SimpleEngine.popen_uci(STOCKFISH_PATH)
    
    # Process test games
    test_moves_df = process_games(test_adhd_games, group_label='Test', engine=engine)
    
    # Display sample results
    print("\nSample of processed moves with complexity scores:")
    print(test_moves_df[['MoveNumber', 'SAN', 'ElocatorComplexity', 'PositionComplexity']].head())
    
    # Basic complexity analysis
    print("\nComplexity Score Statistics:")
    print(test_moves_df['ElocatorComplexity'].describe())
    
    # Plot complexity distribution
    plt.figure(figsize=(10, 6))
    sns.histplot(data=test_moves_df, x='ElocatorComplexity', bins=20)
    plt.title('Distribution of Position Complexity Scores')
    plt.show()
    
    # Cleanup
    engine.quit()
else:
    logging.warning("No test games were fetched.")